# From Zero to Hero in Collateral Shortfall Monitoring

### Introduction

In this notebook, we will showcase how quickly a dashboard can be put together for a simplified use case of Collateral Shortfall monitoring with Atoti libraries.  
  
Collateral is a form of credit risk mitigation where an asset is accepted as security for extending a loan. 
Collateral Shortfall occurs when the collateral value goes below the cash out value. That meant that the value of collateral is less that what it is expected to be, due to a variety of factors such as market fluctuations, contracts enforceability etc. 

We will be creating a multi-dimension data cube and derive the various measures such as market value, collateral value after haircut, cash out value over account and thereafter, the Collateral shortfall for the accounts. 

Leveraging on the data cube and Atoti's data visualization, we will put together dashboards that reflects collateral status of accounts. Cherry on top, we will perform some *What if Analysis* to demonstrate the impact on Collateral in the below scenarios:  

- Haircut simulation where there is a consumption slowdown in the Consumer Cyclical sector in Italy and France
- Technology slump where price value for the sector dipped by 30%
- Cash out simulation where cash out increase by 10%

In [ ]:
import atoti as tt
import pandas as pd
# conda install python-wget if you haven't installed wget
import wget

# last run version: '1.0.3'
pd.__version__

In [ ]:
current_version = tt.__version__
used_version = '0.3.0.20200325172102'

# In the event a newer version of Atoti is used, please update the necessary functions according to the changelog
if used_version != current_version:
    raise ValueError('Version mismatch')

In [ ]:
# downloading data files for this notebook - skip if you already have
wget.download('http://data.atoti.io/notebooks/collateral-shortfall-monitoring/assets_positions.csv')
wget.download('http://data.atoti.io/notebooks/collateral-shortfall-monitoring/assets_prices.csv')
wget.download('http://data.atoti.io/notebooks/collateral-shortfall-monitoring/assets_attributes.parquet')
wget.download('http://data.atoti.io/notebooks/collateral-shortfall-monitoring/loans_positions.csv')

In [ ]:
#tell atoti to load the database containing the UI dashboards
config = tt.config.create_config(metadata_db="./metadata.db")

# a session has to be created for atoti
session = tt.create_session(config=config)

### Data loading
session is used to read data of formats csv, parquet, pandas dataframe, numpy and spark   
refer to https://www.atoti.io/documentation/tutorial/08-data-sources.html   
   
#### Loading csv

In [ ]:
asset_positions_store = session.read_csv('assets_positions.csv', keys=['Account','Asset_Code'], store_name='asset_positions_store')

In [ ]:
# use store.head(n) preview the first n rows loaded into the store where needed
# likewise, you can use store.columns to preview the columns in the store
# or store.shape to see the number of rows and columns loaded into the store
asset_positions_store.head(5)

In [ ]:
assets_prices = session.read_csv('assets_prices.csv', keys=['Asset_Code','Date'], store_name='assets_prices_store')

### Loading parquet

In [ ]:
assets_store = session.read_parquet('assets_attributes.parquet', keys=['Asset_Code'], store_name='assets_store')

### Loading csv via pandas before session loads pandas dataframe   
Being able to load pandas dataframe allows us flexibility of data manipulations using dataframe in the way we are used to.  
This also meant that existing programs using dataframe to consume data can easily be migrated over to Atoti.

In [ ]:
loans_positions_df =pd.read_csv('loans_positions.csv')

In [ ]:
loans_positions_store = session.read_pandas(loans_positions_df, keys=['Account'], store_name='loans_positions_store')

### Joining datastore

In [ ]:
asset_positions_store.join(assets_store, mapping={'Asset_Code' : 'Asset_Code'})

In [ ]:
asset_positions_store.join(loans_positions_store, mapping={'Account' : 'Account'})

In [ ]:
# take note that we are slicing the data by 'Date' column. 
# this means that we always view a subset of the cube by one date by default
asset_positions_store.join(assets_prices, mapping={'Asset_Code' : 'Asset_Code'}, slicing_columns=['Date'])

In [ ]:
# Use head(joined_columns=True) to view the results of the joined references
asset_positions_store.head(joined_columns=True)

### Cube creation

In [ ]:
asset_positions_cube = session.create_cube(asset_positions_store, 'Collateral_Management')

### Quick analysis with cube.visualize

In [ ]:
# we can perform drill-down to different hierarchies in a pivot table
asset_positions_cube.visualize()

In [ ]:
# we can look at the price.VALUE across Date, further split the charts by Asset_Code
asset_positions_cube.visualize()

In [ ]:
# give a meaningful title to the visualization. This helps to reconcile the objective of the visual and also could be the title of 
# the widget when visual is published
asset_positions_cube.visualize('times-series')

In [ ]:
asset_positions_cube.visualize('explore-dataset-using-pivot-table')

### Cube structure
During cube creation, numeric values are automatically created as measures. Non numeric values are automatically translated to levels under hierarchy of the same name. This can be [configured](https://www.atoti.io/documentation/lib/atoti.html#atoti.session.Session.create_cube) differently.

In [ ]:
m = asset_positions_cube.measures
h = asset_positions_cube.hierarchies
lvl = asset_positions_cube.levels

In [ ]:
m

### Creating new measures  
From the data we have, we can derive the following:   
$Market Value = Price \times Quantity$  
$Collateral Value = Market Value \times (1 - Haircut)$   
  
The above measures are aggregated over the Account and Asset Code levels in order to compute the Collateral Shortfall at account level:  
$Collateral Shortfall = Collateral Value - Cash Out$   
Where Cash Out is also aggregated at account level"            

In [ ]:
m['Market Value'] = tt.agg.sum(m['Price.VALUE'] * m['Quantity.SUM'], scope=tt.scope.origin('Account', 'Asset_Code'))

In [ ]:
m['Collateral Value'] = tt.agg.sum(m['Price.VALUE'] * m['Quantity.SUM'] * (1-m['Haircut.VALUE']), tt.scope.origin('Account', 'Asset_Code'))

In [ ]:
m['Cash Out'] = tt.agg.sum(m['Cash_Out.VALUE'], tt.scope.origin('Account'))

In [ ]:
m['Collateral Shortfall'] = m['Collateral Value'] - m['Cash Out']

In [ ]:
m

### Monitoring Collateral Shortfall  
We create a pivot table for with the Collateral Shortfall, Market Value, Cash Out and Collateral Value for Accounts.  
Negative Collateral Shortfall were highlighted in red. Feel free to click on the `>` to drill-down to other hierarchies such as Sector to account for the shortfall. 

In [ ]:
asset_positions_cube.visualize('Collateral Shortfall')

In view of all accounts, using a Gauge chart will show us that we are not yet in shortfall and how far we are from it.   
The red marker shows the total Market value, which is the maximum threshold before shortfall will occur.

In [ ]:
asset_positions_cube.visualize('total-cash-out')

### Total Cash out Bank wide  
  
We can use a Tree map to visualize the asset concentration. A well diversified portfolio will help to reduce the collateral risks.

In [ ]:
asset_positions_cube.visualize('Asset_Concentration')

## Atoti UI and Dashboard creation
Until now, we have created a few visualizations. We can right-click on the visuals to publish them as widgets.
These widgets can then be used to build a dashboard.  

<img src="http://data.atoti.io/notebooks/collateral-shortfall-monitoring/collateral_dashboard.gif" alt="collateral_dashboard" style="zoom:40%;" />

In [ ]:
from IPython.display import Markdown as md 
md("[" + session.url + "/#/dashboard/d13](" + session.url + "/#/dashboard/d13)")

Click on the URL above to view the dashboard that was prepared. We can use the quick filter to select an account for viewing.  
You can also do a right-click drillthrough to investigate the underlying data.  
  
If you wish to play with the UI and explore the data by yourself, [you can have a look at our UI documentation here](https://www.activeviam.com/activeui/documentation/index.html).

## Simulations
Now that we have basic monitoring on Collateral Shortfall, we can do some simulations in the data cube without having to duplicate or reload data.  
  
### Setup Haircut Simulation
We are going to perform a Haircut simulation where there is a consumption slowdown in the Consumer Cyclical sector in Italy and France.  
In this use case, we will replace the value of haircut for the mentioned sector and country to see its downstream impact on the Collaterals.

In [ ]:
# we have to do a setup_simulation, where scenarios added to this simulation will follow the same pattern of simulation (E.g. same levels and method on measure, in this case, replace Haircut) 
# we set the base_scenario_name to "Current Market Condition" - this is the original set of data currently in the cube 
haircut_simulation = asset_positions_cube.setup_simulation("Haircut Simulation", per=[lvl["Sector"],lvl["Country"]], replace=[m["Haircut.VALUE"]], base_scenario_name = "Current Market Condition")

In [ ]:
# create a new scenario with a meaningful name
consumption_slowdown = haircut_simulation.scenarios['Consumption Slowdown']

In [ ]:
# parameters in the sequence of levels defined in the "per" during setup_simulation 
consumption_slowdown += ("Consumer Cyclical", "Italy", 0.20, tt.simulation.Priority.CRITICAL) 
consumption_slowdown += ("Consumer Cyclical", "France", 0.15, tt.simulation.Priority.CRITICAL)

#### Analysing the impact of Consumption Slowdown  
  
We create a chart showing Collateral Shortfall against Account, split by *Haircut Simulation* - the Measure Simulation we just created.  
We will be able to see that with a slowdown in Italy ad France, *Gates & Melinda* will go into shortfall and the shortfall amount for *Buffet* increased dramatically.   
The Consumer Cyclical  in Italy and France does not impact *Daniel EK*, *Musk* and *Niel*.

In [ ]:
asset_positions_cube.visualize('Consumption Slowdown')

With a pivot table, we can easily see the amount of collateral generated by the *Consumer Cyclical* sector in Italy and France.

In [ ]:
asset_positions_cube.visualize('Consumption Slowdown by Sector')

### Market Value Simulation
We are going to create a new set of simulation where technology sector goes into a slump and price value for the sector dipped by 30%.  
We see that while the levels we will be simulating on is the same as the previous simulation, now instead of replacing the Haircut value, we will multiply the Price value by 0.7.  

In [ ]:
market_value_simulation = asset_positions_cube.setup_simulation("Market Value Simulation", per=[lvl["Sector"],lvl["Country"]], multiply=[m["Price.VALUE"]], base_scenario_name = "Current Market Condition")

In [ ]:
tech_bubble = market_value_simulation.scenarios["Tech Bubble ?"]

In [ ]:
# we are going to simulate a slump technology in all countries, therefore we pass the value `*` for country level
tech_bubble += ("Technology", "*", 0.70, tt.simulation.Priority.CRITICAL)

#### Analysing the impact of Technology slump  
  
As before, we plot a chart of Collateral Shortfall against Account. However, we are going to do a split by the *Market Simulation* that we just created.  
We will see the technology giant such as *Gates & Melinda* suffering greatly as their shortfall goes beyond 1.5 million.  
Likewise Buffet with 1/6 of assets in Technology also suffered.

In [ ]:
asset_positions_cube.visualize("Tech Bubble ?")

As we drill-in further to simulation for *Buffet* vs *Gates*, we see that *Buffet* has greater shortfall as they have a higher Cash Out value for the Technology sector than *Gates* while their market value was the same. 

In [ ]:
asset_positions_cube.visualize("Market Simulation - Gates vs Buffet")

### Setup Cash out Simulation

The previous simulations we have done is called Measure simulations. We will now create a different kind of simulation call Source simulation. 
Source simulation as the name suggest, we manipulate the source data before uploading into the store.  
In this case, we take the loans_positions_df that was loaded into the cube previously and modify the Cash_Out value.

This is a Cash out simulation where cash out value increases by 10%.

In [ ]:
loans_positions_df["Cash_Out"] = loans_positions_df["Cash_Out"].apply(lambda x: (1+0.1)*x)

In [ ]:
loans_positions_df

In [ ]:
# In Source Simulation, we do not perform simulation_setup. We just load it to the store as scenarios.
predictive_simulation = loans_positions_store.scenarios["Cash out Increase"].load_pandas(loans_positions_df, truncate=True)

#### Analysing the impact of increase in Cash Out

While cash out increases but collateral value remains the same, we see that the shortfall situation is worsen.

In [ ]:
asset_positions_cube.visualize("Cash out Increase")

#### Activate Source Simulation

In Atoti UI, we have 2 kinds of widgets for simulations:  

- Measure Simulations
  - enable user to update, create new or duplicate simulations
  - available upon setup_simulation
- Source Simulations
  - enable users to upload their modified source files via UI as simulation
  - have to explicitly enable to be accessible from UI
  
Next we will enable source simulation.

In [ ]:
loans_positions_store.source_simulation_enabled = True

Once you have enabled the source simulation, you can proceed to integrate the Source Simulations widget in your dashboard.  
<img src="http://data.atoti.io/notebooks/collateral-shortfall-monitoring/source_simulation.gif" alt="source_simulation" style="zoom:40%;" />

In [ ]:
# this command gives the URL to the Atoti UI for the session
session.url